<a href="https://colab.research.google.com/github/elemnurguner/data-ai-projects/blob/main/Kamera_G%C3%B6r%C3%BCnt%C3%BClerini_Analiz_Eden_AI_Ajan%C4%B1_%C3%96rne%C4%9Fi_(Google_Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python numpy matplotlib tensorflow

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, Javascript, Image
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from google.colab.output import eval_js
# Önceden eğitilmiş modeli yükle
model = MobileNetV2(weights='imagenet')

def analyze_image(img_array):
    # Görüntüyü model için hazırla
    img = cv2.resize(img_array, (224, 224))
    img = preprocess_input(np.expand_dims(img, axis=0))

    # Tahmin yap
    predictions = model.predict(img)
    decoded_predictions = decode_predictions(predictions, top=3)[0]

    # En yüksek skorlu tahmini al
    best_pred = decoded_predictions[0]
    label = best_pred[1]
    confidence = best_pred[2]

    return label, confidence

def make_decision(label, confidence):
    """AI ajanının karar verme mekanizması"""
    if confidence < 0.5:
        return "Belirsiz nesne - Eyleme geçmiyorum"

    # Örnek karar kuralları
    if label == 'dog':
        return "Köpek tespit edildi! Kapıyı açıyorum."
    elif label == 'cat':
        return "Kedi tespit edildi! Mama kabını dolduruyorum."
    elif label == 'car':
        return "Araba tespit edildi! Garaj kapısını açıyorum."
    else:
        return f"{label} tespit edildi. Özel bir eylem tanımlı değil."

# Colab'da kamera erişimi için JavaScript kodu
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))
    # Get OpenCV format image
    img = js_to_image(data)

    return img

def js_to_image(js_reply):
    """JavaScript tarafından alınan görüntüyü OpenCV formatına çevirir"""
    import numpy as np
    import base64
    import re

    # Base64 kodunu çöz
    image_bytes = base64.b64decode(js_reply.split(',')[1])
    # Görüntüyü numpy array'e çevir
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # OpenCV formatında aç
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

# Ana işlem
try:
    print("Kameraya erişim sağlanıyor...")
    img = take_photo()
    print("Fotoğraf çekildi. Analiz başlıyor...")

    # Görüntüyü analiz et
    label, confidence = analyze_image(img)
    print(f"Tespit edilen nesne: {label} (%{confidence*100:.2f} doğruluk)")

    # Karar ver
    decision = make_decision(label, confidence)
    print("AI Ajanı Kararı:", decision)

    # Sonuçları görselleştir
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"Tespit: {label} (%{confidence*100:.2f})\nKarar: {decision}")
    plt.axis('off')
    plt.show()

except Exception as err:
    print(f"Hata oluştu: {str(err)}")
    print("Not: Bu kodun çalışması için Google Colab'da çalıştırılması ve kamera erişimine izin verilmesi gerekiyor.")